<a href="https://colab.research.google.com/github/toinnn/Torax_Xray-Health-Care/blob/main/NIH_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Obtém o número de núcleos da CPU
num_cores = os.cpu_count()

# Obtém informações sobre a CPU
cpu_info = !lscpu

# Imprime o número de núcleos e informações sobre a CPU
print(f"Número de núcleos da CPU: {num_cores}")
# print("Informações sobre a CPU:")
# for line in cpu_info:
#     print(line)

Número de núcleos da CPU: 2


In [3]:
Git_Dir    = "/content/drive/MyDrive/Github_Dir/Torax_Xray-Health-Care" #https://github.com/toinnn/Torax_Xray-Health-Care
Neural_Dir = "Torax_Xray-Health-Care"
with open("/content/drive/MyDrive/Github_Dir/acess_Token_Git.txt","r") as file:
    acess_Token_Git = file.read()
Git_Path   = "https://"+ acess_Token_Git + "@github.com/toinnn/" + Neural_Dir + ".git"
# Git_CB_Path= "https://"+ acess_Token_Git + "@github.com/toinnn/" + "Chat_Bot" + ".git"

In [4]:
!git clone "{Git_Path}" ./temp/Torax_Xray-Health-Care
# !git clone "{Git_CB_Path}" ./temp/Chat_Bot

!mv ./temp/* "{Git_Dir}"
!rm -rf ./temp

!rsync -aP "{Git_Dir}"/*  ./
# #!ln -s "/content/drive/MyDrive/Github_Dir/Chat_Bot" + Neural_Dir NLP

Cloning into './temp/Torax_Xray-Health-Care'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 1.21 MiB | 38.67 MiB/s, done.
Resolving deltas: 100% (129/129), done.
mv: inter-device move failed: './temp/Torax_Xray-Health-Care' to '/content/drive/MyDrive/Github_Dir/Torax_Xray-Health-Care/Torax_Xray-Health-Care'; unable to remove target: Directory not empty
sending incremental file list
Torax_Xray-Health-Care/


In [5]:
%cd "/content/{Neural_Dir}"
# %cd "/content/drive/MyDrive/Github_Dir/Torax_Xray-Health-Care"

# %cd /content/Chat_Bot

/content/Torax_Xray-Health-Care


In [6]:

def image_i_path(id):
    if id <= 4999 :
        i = 1
    elif id <=  14999 :
        i = 2
    elif id <=  24999 :
        i = 3
    elif id <=  34999 :#24999 + 10000:
        i = 4
    elif id <=  44999 : #24999 + 9829:
        i = 5
    elif id <=  54999 : #24999 + 9829:
        i = 6
    elif id <=  64999 : #24999 + 9829:
        i = 7
    elif id <=  74999 : #24999 + 9829:
        i = 8
    elif id <=  84999 : #24999 + 9829:
        i = 9
    elif id <=  94999 : #24999 + 9829:
        i = 10
    elif id <= 104999 : #24999 + 9829:
        i = 11
    elif id <= 104999 + 7121 : #24999 + 9829:
        i = 12
    if i < 10 :
      return  f"/content/drive/MyDrive/X-ray_2/Dataset/images_00{i}/images"
    else :
      return  f"/content/drive/MyDrive/X-ray_2/Dataset/images_0{i}/images"

In [7]:
from Transformer_Decoder import decoder , Trainer
import torch
import torch.nn as nn
# from Transformer_Decoder import decoder , Trainer
from torch.utils.data import DataLoader , Dataset
from torch.nn.utils.rnn import pad_sequence
import json
# from torchvision.io.image import decode_png, read_image
from torchvision.io import decode_png, read_image , ImageReadMode
import torchvision.transforms as T
import polars as pl
from PIL import Image
import numpy as np
import pickle


def load_image_nvjpngl_gpu(image_path):
    """
    Loads an image from the specified file path using NVJPEG decoder on GPU and returns a PyTorch tensor.
    Args:
        image_path (str): The path to the image file.
    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    data = read_image(image_path , mode = ImageReadMode.GRAY )
    return data.float()
    # print(data)
    tensor = decode_png(data).float()#.to("cuda")
    return tensor
def meu_collate(batch):
    entradas, saidas = zip(*batch)
    # Pad nas sequências de entrada
    entradas_padded = pad_sequence(entradas, batch_first=True, padding_value=0)

    # Pad nas sequências de saída
    saidas_padded = pad_sequence(saidas, batch_first=True, padding_value=0)

    return entradas_padded, saidas_padded
    # return pad_sequence([a, b, c])
class dataset_NIH_Chest(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs : list[str] , Data_Entry_path : str , image_dir_path : str , label2class_path : str , max_label_lengh : int ):
        'Initialization'
        # self.labels = labels
        self.list_IDs = tuple(list_IDs) #Lista contendo os nomes das imagens
        self.image_dir_path = image_dir_path #string com o path da pasta que contem as imagens de input
        self.Data_Entry  = pl.read_csv(Data_Entry_path)
        self.label2class = json.load(open(label2class_path , "rb"))
        # self.y_len_max = max_label_lengh + 1
        # for row in data_intro[['Image Index' ,'Finding Labels' ]].iter_rows(named=True) :
        #     print({row['Image Index'] : row['Finding Labels'].split("|") })
        #     raise
        self.Data_Entry = {row['Image Index'] : row['Finding Labels'].split("|") for row in self.Data_Entry[['Image Index' ,'Finding Labels' ]].iter_rows(named=True)}
        """for row in data_intro[['Image Index' ,'Finding Labels' ]].iter_rows(named=True) :
            if max_label_lengh < len( row['Finding Labels'].split("|")  )  : """
        # keys = tuple(self.Data_Entry.keys())
        # self.name_2_index = { keys[i] : i+1  for i in range(len(keys)) }


    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        # linha_encontrada = self.Data_Entry.filter(( self.Data_Entry['Image Index'] == self.list_IDs[index] ) )
        # linha_encontrada['Finding Labels'].split("|")

        y = torch.tensor([ int(self.label2class[i]) for i in self.Data_Entry[ID]] ).view(-1, 1) #linha_encontrada['Finding Labels'][0].split("|") ]).view(-1, 1)



        # aux = torch.zeros( self.y_len_max - y.shape[0]  , 1)
        aux = torch.zeros( 1  , 1)
        y = torch.cat([y , aux] , dim = 0)

        # Load data and get label
        X = load_image_nvjpngl_gpu(self.image_dir_path + self.list_IDs[index]) #torch.load( self.image_dir_path + self.list_IDs[index] )


        # path = image_i_path( self.name_2_index[ID] ) + "/" + ID
        # X = load_image_nvjpngl_gpu(path)


        return X, y


class my_model(nn.Module):
    def __init__(self ,  device : torch.device = torch.device("cpu")) -> None:
        super(my_model , self  ).__init__()
        vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
        # vits14.eval()
        # print(vits14.eval())
        self.encoder = vits14.to(device) #EU NÃO LEMBRO QUANTO DEVERIA SER O MODEL_DIM !!!!!
        self.decoder = decoder(model_dim = 768 ,heads = 8 ,num_layers = 8 , num_Classes = 16 , device = device)
        self.device  = device
        # self.transform_image = T.Compose([T.Resize(244), T.CenterCrop(224), T.Normalize([0.5], [0.5])] )
        self.transform_image = T.Compose([T.Resize(224),  T.Normalize([0.5], [0.5])] )

    def setDevice(self , device : torch.device) :
        self.encoder = self.encoder.to( device )
        self.decoder.setDevice( device )
        return


    def forward_fit(self, image  , max_lengh = 100):
        # print(f"Passou do Encoder image : { image.shape}")
        img2 = []
        for img in image :
            img = img.view(1, img.shape[0] , img.shape[1])
            img = torch.cat( [img,img,img] , dim = 0 )
            # print(f"img.shape = {img.shape}")
            # print(f"trasnform : {self.transform_image( img  )[:3].unsqueeze(0).shape }")
            img2 += [self.transform_image( img )[:3].unsqueeze(0) ]
        # image = [ self.encoder(self.transform_image( img.view(1 , img.shape[0] , img.shape[1]))[:3].unsqueeze(0) ).view(1,1,-1)     for img in image]
        image = [ self.encoder(img).view(1 , 1 , -1) for img in img2 ]
        enc   = torch.cat(image , dim = 0 )
        # image = self.transform_image(image)[:3].unsqueeze(0)
        # print(f"Passou do Encoder image : {image.shape}")
        # enc = self.encoder(image)
        # print(f"Passou do Encoder enc : {enc.shape}")
        return self.decoder.forward_fit(enc , enc , max_lengh)

    def forward(self, image  , max_lengh = 100):
        # image = self.transform_image(image)[:3].unsqueeze(0)
        # enc = self.encoder(image)

        img2 = []
        for img in image :
            img = img.view(1, img.shape[0] , img.shape[1])
            img = torch.cat( [img,img,img] , dim = 0 )
            # print(f"img.shape = {img.shape}")
            # print(f"trasnform : {self.transform_image( img  )[:3].unsqueeze(0).shape }")
            img2 += [self.transform_image( img )[:3].unsqueeze(0) ]
        # image = [ self.encoder(self.transform_image( img.view(1 , img.shape[0] , img.shape[1]))[:3].unsqueeze(0) ).view(1,1,-1)     for img in image]
        image = [ self.encoder(img).view(1 , 1 , -1) for img in img2 ]
        enc   = torch.cat(image , dim = 0 )

        return self.decoder(enc , enc , max_lengh)




In [10]:
if __name__ == '__main__':

    max_label_lengh = 9
    params = {'batch_size': 30   ,
        'shuffle'         : True ,
        'num_workers'     : 8    ,
        "collate_fn"      : meu_collate}

    path_project  = "/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray"
    image_dir_path = path_project + "/images/"
    data_intro_path = path_project + "/Data_Entry_2017.csv"
    label2class_path = path_project + "/target_dict_label2class.json"



    train_path = path_project + "/train_val_list.txt"
    test_path  = path_project + "/test_list.txt"

    train_id = [i.replace("\n" , "") for i in open(train_path , "r").readlines() ][:330]
    test_id  = [i.replace("\n" , "") for i in open(test_path , "r" ).readlines() ][:350]

    training_set = dataset_NIH_Chest(train_id , data_intro_path , image_dir_path  , label2class_path , max_label_lengh )
    training_loader = DataLoader(training_set  , **params)

    test_set = dataset_NIH_Chest(test_id , data_intro_path , image_dir_path  , label2class_path , max_label_lengh )
    test_loader = DataLoader(test_set, **params)




    model   = my_model(torch.device("cuda"))
    trainer = Trainer(model , torch.device("cuda") )
    model , loss  = trainer.fit(training_loader  , 0.05 , 1 , 1 , test_dataloader = test_loader )

    path_2_counter = "/content/drive/MyDrive/Github_Dir/models_saved.json"

    with open(path_2_counter , "r") as arquivo_json:
        counter = json.load(arquivo_json)
    new_id = int(counter["ID_atual"])

    path_2_save = f"/content/drive/MyDrive/Github_Dir/xRay_model_{new_id}_loss_{loss}.model"

    try:
        with open(path_2_save, "wb") as arquivo:
            pickle.dump(model, arquivo)
    except FileNotFoundError:
        with open(path_2_save, "xb") as arquivo:
            pickle.dump(model, arquivo)

    counter["ID_atual"] = f"{new_id + 1}"

    with open(path_2_counter , "w") as arquivo_json:
        json.dump(counter , arquivo_json)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:

chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
chegou aqui
Age atual 0


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


 ctd atual 0  samples processados 0
out.shape = torch.Size([30, 16, 5]) , y.shape = torch.Size([30, 5])
loss : 0.01830989308655262
Pré backward
Pós backward
 ctd atual 1  samples processados 30
out.shape = torch.Size([30, 16, 4]) , y.shape = torch.Size([30, 4])
loss : 0.023540765047073364
Pré backward
Pós backward
lossTestList : [0.03428571428571429]
bestLossValue = inf
Novo melhor
Objeto salvo em best_model_in_test.pickle
Saiu do Melhor
 ctd atual 2  samples processados 60
out.shape = torch.Size([30, 16, 5]) , y.shape = torch.Size([30, 5])
loss : 0.01876162923872471
Pré backward
Pós backward
 ctd atual 3  samples processados 90
out.shape = torch.Size([30, 16, 4]) , y.shape = torch.Size([30, 4])
loss : 0.018330121412873268
Pré backward
Pós backward
lossTestList : [0.03428571428571429, 0.02991746031746032]
bestLossValue = 0.03428571428571429
Novo melhor
Objeto salvo em best_model_in_test.pickle
Saiu do Melhor
 ctd atual 4  samples processados 120
out.shape = torch.Size([30, 16, 5]) , y.

In [9]:
load_image_nvjpngl_gpu('/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray/images/00005762_004.png')
# load_image_pil_accelerated('/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray/images/00005762_004.png')

tensor([[[  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         ...,
         [ 97., 163., 152.,  ...,  44.,  46.,  26.],
         [106., 179., 165.,  ...,  47.,  49.,  27.],
         [ 53.,  89.,  81.,  ...,  23.,  23.,  13.]]])

In [ ]:
len(test_id)

25596

In [ ]:
vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:02<00:00, 169MB/s]


In [ ]:
import pickle


path = "/content/xRay_model_1_loss_0.03098666666666666.model"
with open(path, 'rb') as file:
    # model = pickle.load(file)
    model = pickle.load(file)


RuntimeError: ignored

In [ ]:
# pip install datasets


In [ ]:
# from datasets import load_dataset

# # Carregando o dataset "alkzar90/NIH-Chest-X-ray-dataset"
# dataset = load_dataset("alkzar90/NIH-Chest-X-ray-dataset", 'image-classification' )

# # Acessando os dados de treinamento
# train_data = dataset["train"]

# # Imprimindo algumas informações
# print(f"Número de exemplos no conjunto de treinamento: {len(train_data)}")
# print(f"Exemplo de texto: {train_data[0]['text']}")
# print(f"Rótulo do exemplo: {train_data[0]['label']}")
